Key Points:

Embedded Documents: Use embedded documents to store related data together, which allows for efficient querying.

The common attribute of these two dataset is date, if we do embedded data model, it would accelerate all the queries related with date, range of hotspot data would be limited to the range with this specific date.

In [1]:
import os
print(os.getcwd())


/home/student/A2


In [2]:
os.chdir("/home/student/A2/drive-download-20240507T073745Z-001")

In [3]:
os.listdir()

['climate_historic.csv',
 'climate_streaming.csv',
 'hotspot_AQUA_streaming.csv',
 'hotspot_historic.csv',
 'hotspot_TERRA_streaming.csv']

In [4]:
import pandas as pd
hotspot = pd.read_csv("./hotspot_historic.csv", sep=',')
climate = pd.read_csv("./climate_historic.csv", sep=',')

In [5]:
hotspot.head()


,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2023-12-27 04:16:51,78,27/12/2023,68
1,-35.541,143.311,2023-12-27 00:02:15,82,27/12/2023,63
2,-35.554,143.307,2023-12-27 00:02:15,67,27/12/2023,53
3,-35.543,143.316,2023-12-27 00:02:14,86,27/12/2023,67
4,-37.708,145.100,2023-12-25 04:29:08,80,25/12/2023,54


In [6]:
climate.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,948700,31/12/2022,19,56.8,7.9,11.1,0.00I,154
1,948700,02/01/2023,15,50.7,9.2,13.0,0.02G,128
2,948700,03/01/2023,16,53.6,8.1,15.0,0.00G,133
3,948700,04/01/2023,24,61.6,7.7,14.0,0.00I,186
4,948700,05/01/2023,24,62.3,7.0,13.0,0.00I,185


In [7]:
climate.tail()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
361,948702,28/12/2023,21,61.1,6.6,11.1,0.00I,163
362,948702,29/12/2023,19,59.7,7.4,14.0,0.63G,150
363,948702,30/12/2023,16,51.5,8.7,15.0,0.02G,135
364,948702,31/12/2023,18,53.6,7.9,15.9,0.00G,150
365,948702,01/01/2024,19,52.9,8.1,15.0,0.00I,159


In [8]:
print(climate['date'].dtype)

object


In [9]:
import csv
from pprint import pprint

In [10]:
def csv_to_dict(filename):
    with open(filename, mode='r', newline='', encoding='utf-8') as csvfile:
        #DictReader read the first row as column headers
        #return an iterator that produce disctionaries for each row
        #keys: column headers
        reader = csv.DictReader(csvfile)
        #convert iterator into list of dictionaries
        #keys: columne headers, values: cell values
        return list(reader)


# Read climate data
climate_data = csv_to_dict('climate_historic.csv')

# Read hotspot data
hotspot_data = csv_to_dict('hotspot_historic.csv')



In [11]:
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = '172.16.62.212'

client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

In [12]:
#avoid error from exsiting collection with the same name
collection.drop()

In [13]:
# Convert the date columns to datetime objects for proper comparison
                        #convert from d/m/Y format into 
# climate['date'] = pd.to_datetime(climate['date'], format='%d/%m/%Y')
# hotspot['date'] = pd.to_datetime(hotspot['date'], format='%d/%m/%Y')

#since the locaiton of the sensors would not change across the dates
#since the station would not change location across the dates
# Embed fire data into climate data
# loop through each entry of the climate 
for index, climate_row in climate.iterrows():
    
    #filter out the climate data entries that have the sensor data
    #since they are important, others are not
    #since stations have different data according to dates

    #for each climate, filter out the hotspot/sensors with matching/corresponding date
    #list of true/false to filter out the hotspots related to the dates
    useful_dates = hotspot['date'] == climate_row['date']
    matching_fires = hotspot[useful_dates]

    #convert to list of disctionaries
    fires_list = matching_fires.to_dict('records')
    
    #inner-join
#     if fires_list:
        #print(fires_list)
    #left-outer join
    climate_doc = {
        'stations': climate_row['station'],
                                    #string from time with format, 
                                    #since MongoDB can not store object
        #.strftime('%d/%m/%Y')
        'date': climate_row['date'],
        'air_temperature_celcius': climate_row['air_temperature_celcius'],
        'relative_humidity': climate_row['relative_humidity'],
        'windspeed_knots': climate_row['windspeed_knots'],
        'max_wind_speed': climate_row['max_wind_speed'],
        'precipitation': climate_row['precipitation'],
        'GHI_w/m2': climate_row['GHI_w/m2'],
        #embed sensor data here
        'hotspots': fires_list
    }

    collection.insert_one(climate_doc)





In [14]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
"""

# Q2.a.
res = collection.find(
    {
    'date': '12/12/2023'
    }
)

for d in res:
    print(d)

{'_id': ObjectId('66420d6d9e89db15968a9f2c'), 'stations': 948702, 'date': '12/12/2023', 'air_temperature_celcius': 19, 'relative_humidity': 55.3, 'windspeed_knots': 6.2, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 156, 'hotspots': [{'latitude': -37.903, 'longitude': 145.25, 'datetime': '2023-12-12 00:45:38', 'confidence': 53, 'date': '12/12/2023', 'surface_temperature_celcius': 44}]}


In [15]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question b
"""

# as long as include filtering/matching on the documents of the hotspots array field in the main document
# use aggregate + pipeline

pipeline = [
    {   #first match, filter the station+date documents based on the hotspots that match
        '$match': {
            'hotspots.surface_temperature_celcius': {
                '$gte': 65,
                '$lte': 100
            }
        }
    },
    {   # since not all sensors in the document indexed by date and station
        # fulifll the condition, use unwind to release documents in array field(hotspots)
        # to flatten array
        '$unwind': '$hotspots'  # Unwind the hotspots array
    },
    {   #second match, filter the hotspot documents of the hotspots array
        '$match': {
            'hotspots.surface_temperature_celcius': {
                '$gte': 65,
                '$lte': 100
            }
        }
    },
    {
        '$project': { #   #value from output document after matching
                            #$ -> latitude field
            'latitude': '$hotspots.latitude',
            'longitude': '$hotspots.longitude',
            'surface_temperature_celcius': '$hotspots.surface_temperature_celcius',
            'confidence': '$hotspots.confidence'
        }
    }
]

res = collection.aggregate(pipeline)

# To display only the latitude
for d in res:
    pprint(d)


{'_id': ObjectId('66420d6d9e89db15968a9eed'),
 'confidence': 96,
 'latitude': -38.127,
 'longitude': 143.82,
 'surface_temperature_celcius': 77}
{'_id': ObjectId('66420d6d9e89db15968a9ef2'),
 'confidence': 100,
 'latitude': -37.294,
 'longitude': 141.232,
 'surface_temperature_celcius': 87}
{'_id': ObjectId('66420d6d9e89db15968a9ef2'),
 'confidence': 99,
 'latitude': -37.293,
 'longitude': 141.245,
 'surface_temperature_celcius': 85}
{'_id': ObjectId('66420d6d9e89db15968a9ef4'),
 'confidence': 91,
 'latitude': -37.247,
 'longitude': 141.278,
 'surface_temperature_celcius': 68}
{'_id': ObjectId('66420d6d9e89db15968a9ef8'),
 'confidence': 92,
 'latitude': -36.098,
 'longitude': 143.74,
 'surface_temperature_celcius': 67}
{'_id': ObjectId('66420d6d9e89db15968a9f0e'),
 'confidence': 89,
 'latitude': -36.94,
 'longitude': 143.281,
 'surface_temperature_celcius': 69}
{'_id': ObjectId('66420d6d9e89db15968a9f10'),
 'confidence': 87,
 'latitude': -37.862,
 'longitude': 144.175,
 'surface_temper

In [16]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question c
"""
#filtering only on main document, no pipeline needed

res = collection.find(
    {
        'date':{
            '$in': ['15/12/2023', '16/12/2023']
        }
    },
    {   #only work for main 
        '_id': 0,
        'date': 1,
        'air_temperature_celcius': 1,
        'relative_humidity': 1,
                                        #must be used for array field
        'surface_temperature_celcius': '$hotspots.surface_temperature_celcius'
    }
)

for d in res:
    pprint(d)


{'air_temperature_celcius': 18,
 'date': '15/12/2023',
 'relative_humidity': 52.0,
 'surface_temperature_celcius': [42, 36, 38, 40]}
{'air_temperature_celcius': 18,
 'date': '16/12/2023',
 'relative_humidity': 53.7,
 'surface_temperature_celcius': [43,
                                 33,
                                 54,
                                 73,
                                 55,
                                 75,
                                 55,
                                 66,
                                 56,
                                 60,
                                 73,
                                 48,
                                 55,
                                 64,
                                 57]}


In [17]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question d
"""

pipeline = [
    {
        '$match': {
            'hotspots.confidence': {
                '$gte': 80,
                '$lte': 100
            }
        }
    },
    {
        '$unwind': '$hotspots'  # Corrected to a string
    },
    {
        '$match': {
            'hotspots.confidence': {
                '$gte': 80,
                '$lte': 100
            }
        }
    }
    # {   # no attributes from main document can be shown with fields of document in array field
    #     '$project': {
    #         'datetime': '$hotspots.datetime',
    #         'surface_temperature_celcius': '$hotspots.surface_temperature_celcius',
    #         'confidence': '$hotspots.confidence'  
    #     }
    # }
]

res = collection.aggregate(pipeline)

for d in res:
    pprint(d)


res = collection.aggregate(pipeline)

for d in res:
    pprint(d)

{'GHI_w/m2': 118,
 '_id': ObjectId('66420d6d9e89db15968a9eed'),
 'air_temperature_celcius': 13,
 'date': '10/10/2023',
 'hotspots': {'confidence': 96,
              'date': '10/10/2023',
              'datetime': '2023-10-10 04:04:19',
              'latitude': -38.127,
              'longitude': 143.82,
              'surface_temperature_celcius': 77},
 'max_wind_speed': 15.0,
 'precipitation': '0.00I',
 'relative_humidity': 42.5,
 'stations': 948702,
 'windspeed_knots': 8.0}
{'GHI_w/m2': 111,
 '_id': ObjectId('66420d6d9e89db15968a9ef2'),
 'air_temperature_celcius': 13,
 'date': '15/10/2023',
 'hotspots': {'confidence': 100,
              'date': '15/10/2023',
              'datetime': '2023-10-15 04:23:01',
              'latitude': -37.294,
              'longitude': 141.232,
              'surface_temperature_celcius': 87},
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 50.7,
 'stations': 948702,
 'windspeed_knots': 6.1}
{'GHI_w/m2': 111,
 '_id': ObjectId

In [18]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question e
"""
                        #sort 
#res = collection.find().sort('surface_temperature_celcius', -1).limit(10)
res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)

for i, d in enumerate(res):
    # only 10 records for res
    # if i == 9:
    pprint(d)

{'GHI_w/m2': 154,
 '_id': ObjectId('66420d6b9e89db15968a9ddb'),
 'air_temperature_celcius': 19,
 'date': '09/01/2023',
 'hotspots': [],
 'max_wind_speed': 8.9,
 'precipitation': '0.00I',
 'relative_humidity': 56.3,
 'stations': 948700,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 161,
 '_id': ObjectId('66420d6b9e89db15968a9ddc'),
 'air_temperature_celcius': 20,
 'date': '10/01/2023',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 57.0,
 'stations': 948700,
 'windspeed_knots': 8.7}
{'GHI_w/m2': 185,
 '_id': ObjectId('66420d6b9e89db15968a9dd7'),
 'air_temperature_celcius': 24,
 'date': '05/01/2023',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 62.3,
 'stations': 948700,
 'windspeed_knots': 7.0}
{'GHI_w/m2': 160,
 '_id': ObjectId('66420d6b9e89db15968a9dda'),
 'air_temperature_celcius': 20,
 'date': '08/01/2023',
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': '0.00I',
 'relative_humidity': 57.7,

In [19]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question f
"""

pipeline = [
    {
        '$group': {
            '_id': '$date',  # group by the 'date' field
            'count': {'$sum': 1}  # count the documents in each group
        }
    }
]

res = collection.aggregate(pipeline)
for i, d in enumerate(res):
    pprint(d)



{'_id': '08/11/2023', 'count': 1}
{'_id': '19/04/2023', 'count': 1}
{'_id': '09/11/2023', 'count': 1}
{'_id': '24/05/2023', 'count': 1}
{'_id': '28/11/2023', 'count': 1}
{'_id': '15/08/2023', 'count': 1}
{'_id': '28/12/2023', 'count': 1}
{'_id': '26/07/2023', 'count': 1}
{'_id': '10/04/2023', 'count': 1}
{'_id': '29/06/2023', 'count': 1}
{'_id': '07/10/2023', 'count': 1}
{'_id': '06/05/2023', 'count': 1}
{'_id': '07/11/2023', 'count': 1}
{'_id': '29/05/2023', 'count': 1}
{'_id': '12/11/2023', 'count': 1}
{'_id': '09/07/2023', 'count': 1}
{'_id': '09/12/2023', 'count': 1}
{'_id': '24/08/2023', 'count': 1}
{'_id': '19/12/2023', 'count': 1}
{'_id': '31/01/2023', 'count': 1}
{'_id': '21/12/2023', 'count': 1}
{'_id': '29/07/2023', 'count': 1}
{'_id': '14/05/2023', 'count': 1}
{'_id': '08/02/2023', 'count': 1}
{'_id': '07/02/2023', 'count': 1}
{'_id': '18/08/2023', 'count': 1}
{'_id': '25/08/2023', 'count': 1}
{'_id': '31/08/2023', 'count': 1}
{'_id': '19/09/2023', 'count': 1}
{'_id': '18/12

In [20]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question g
"""

#Fires records = fields from array field -> pipeline
pipeline = [
    {
        '$unwind': '$hotspots'  # Unwind the hotspots array
    },
    {
        '$match': {
            'hotspots.confidence': {
                '$lt': 70  # Match documents where confidence is less than 70
            }
        }
    },
    {
        '$project': {
            #Just Fire records
            '_id': 0,
            'confidence': '$hotspots.confidence',  
            'date': '$hotspots.date',  
            'datetime': '$hotspots.datetime',  
            'latitude': '$hotspots.latitude',  
            'longitude': '$hotspots.longitude', 
            'surface_temperature_celcius': '$hotspots.surface_temperature_celcius'
        }
    }
]

res = collection.aggregate(pipeline)
for i, d in enumerate(res):
    pprint(d)




{'confidence': 62,
 'date': '16/10/2023',
 'datetime': '2023-10-16 23:56:43',
 'latitude': -37.288,
 'longitude': 144.39,
 'surface_temperature_celcius': 36}
{'confidence': 58,
 'date': '17/10/2023',
 'datetime': '2023-10-17 04:10:38',
 'latitude': -37.587,
 'longitude': 142.479,
 'surface_temperature_celcius': 41}
{'confidence': 65,
 'date': '20/10/2023',
 'datetime': '2023-10-20 00:27:12',
 'latitude': -37.296,
 'longitude': 144.386,
 'surface_temperature_celcius': 54}
{'confidence': 65,
 'date': '21/10/2023',
 'datetime': '2023-10-21 01:09:54',
 'latitude': -36.1,
 'longitude': 143.772,
 'surface_temperature_celcius': 35}
{'confidence': 67,
 'date': '26/10/2023',
 'datetime': '2023-10-26 04:04:53',
 'latitude': -36.156,
 'longitude': 143.568,
 'surface_temperature_celcius': 42}
{'confidence': 69,
 'date': '26/10/2023',
 'datetime': '2023-10-26 04:04:53',
 'latitude': -36.152,
 'longitude': 143.578,
 'surface_temperature_celcius': 44}
{'confidence': 62,
 'date': '27/10/2023',
 'datet

In [21]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question h
"""

pipeline = [
    {
        '$unwind': '$hotspots' 
    },
    {
        '$group': {
            #_id must exist in $group can be replaced
            '_id': '$hotspots.date',  # group by the date field from hotspots
            'avg_surface_temperature': {
                '$avg': '$hotspots.surface_temperature_celcius' 
            }
        }
    },
    {   #modify the project of the output after all above processing
        '$project': {
            '_id': 0,  # Exclude the _id field
            'date': '$_id',  # Map the _id field (which is the date) to 'date' in the output
            'avg_surface_temperature': 1  # 
        }
    },
    {
        '$sort': {'date': 1}  # Optional: sort by date
    }
]

res = collection.aggregate(pipeline)
for d in res:
    print(d)


{'avg_surface_temperature': 53.333333333333336, 'date': '10/10/2023'}
{'avg_surface_temperature': 46.0, 'date': '10/12/2023'}
{'avg_surface_temperature': 46.25, 'date': '11/11/2023'}
{'avg_surface_temperature': 53.0, 'date': '12/11/2023'}
{'avg_surface_temperature': 44.0, 'date': '12/12/2023'}
{'avg_surface_temperature': 47.0, 'date': '13/11/2023'}
{'avg_surface_temperature': 60.0, 'date': '13/12/2023'}
{'avg_surface_temperature': 52.0, 'date': '14/11/2023'}
{'avg_surface_temperature': 70.0, 'date': '14/12/2023'}
{'avg_surface_temperature': 72.66666666666667, 'date': '15/10/2023'}
{'avg_surface_temperature': 39.0, 'date': '15/12/2023'}
{'avg_surface_temperature': 36.0, 'date': '16/10/2023'}
{'avg_surface_temperature': 57.8, 'date': '16/12/2023'}
{'avg_surface_temperature': 51.6, 'date': '17/10/2023'}
{'avg_surface_temperature': 52.166666666666664, 'date': '18/10/2023'}
{'avg_surface_temperature': 50.0, 'date': '20/10/2023'}
{'avg_surface_temperature': 51.25, 'date': '21/10/2023'}
{'avg

In [22]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question i
"""
# sort all by field 'GHI_w/m2' count for 10, -1 decending
res = collection.find().sort('GHI_w/m2', -1).limit(10)

for d in res:
    pprint(d)

{'GHI_w/m2': 265,
 '_id': ObjectId('66420d6b9e89db15968a9dd9'),
 'air_temperature_celcius': 32,
 'date': '07/01/2023',
 'hotspots': [],
 'max_wind_speed': 19.0,
 'precipitation': '0.00I',
 'relative_humidity': 54.1,
 'stations': 948700,
 'windspeed_knots': 12.8}
{'GHI_w/m2': 243,
 '_id': ObjectId('66420d6d9e89db15968a9f10'),
 'air_temperature_celcius': 28,
 'date': '14/11/2023',
 'hotspots': [{'confidence': 87,
               'date': '14/11/2023',
               'datetime': '2023-11-14 04:35:04',
               'latitude': -37.862,
               'longitude': 144.175,
               'surface_temperature_celcius': 65},
              {'confidence': 64,
               'date': '14/11/2023',
               'datetime': '2023-11-14 00:21:26',
               'latitude': -38.527,
               'longitude': 143.493,
               'surface_temperature_celcius': 45},
              {'confidence': 64,
               'date': '14/11/2023',
               'datetime': '2023-11-14 00:21:07',
          

In [23]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question j
"""
#main document aggregate piepeline
pipeline = [
    {
        '$match':{
            'precipitation':{
                '$gte': 0.2,
                '$lte': 0.35
            }
        }
    },
    {
        '$project':{   
            '_id': 0,
            'date': 1,
            'precipitation': 1
        }
    }
]

res = collection.aggregate(pipeline)
for d in res:
    pprint(d)


In [24]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question 3
"""

# Assuming 'db' is your MongoDB database object and 'climate_data' is your collection.
db.climate_data.create_index([
    ('date', pymongo.ASCENDING),
    ('station', pymongo.ASCENDING)
])


'date_1_station_1'

# since there is station for each day, with the compound index of date and station, we can efficiently access the data from climate, and the hotspots related to that date are limited, which can be further filtered with the locations of the hotspots by clutering.

In [25]:
# # Query the collection using the compound index

# All documents for station 948700 on January 1, 2023
results = collection.find({
    'date': '31/12/2023',
    'station': 948700
})

# Process the results
for doc in results:
    pprint(doc)



In [26]:
"""
Part b
Task 1. Processing Data Stream (45%)
Question a
"""
from datetime import datetime, timedelta

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

2024-01-01 00:00:00


In [38]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json

hostip = "172.16.62.212" # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer
    
def publish_msg(producer, topic, data):
    try:
        #print(data)
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [ ]:
"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Scenaries01'
    print('Publishing records..')
    producer01 = connect_kafka_producer()


    # Assuming 'climate' is your DataFrame
    #print(len(climate.index))
    rand_row_indices = list(range(len(climate.index)))
    random.shuffle(rand_row_indices)  
    #print(len(indices))

    for index, climate_row in climate.iterrows():
        climate_row = climate.loc[rand_row_indices[index]]

        #inner-join
    #     if fires_list:
            #print(fires_list)

        #left-outer join
        #print(start_date.strftime('%Y-%m-%d'))
        climate_doc = {
            'stations': str(climate_row['station']),
                                        #string from time with format, 
                                        #since MongoDB can not store object
            #.strftime('%d/%m/%Y')
            #convert datetime object to string in format
            # laster dumps not work on datetime object but number of string
            'date': start_date.strftime('%Y-%m-%d'),
            'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
            'relative_humidity': float(climate_row['relative_humidity']),
            'windspeed_knots': float(climate_row['windspeed_knots']),
            'max_wind_speed': float(climate_row['max_wind_speed']),
            'precipitation': str(climate_row['precipitation']),
            'GHI_w/m2': int(climate_row['GHI_w/m2']),
            'producer': "Producer_1"

        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        publish_msg(producer01, topic, climate_doc)
        start_date += timedelta(days=1)
        sleep(10) # every 10 seconds send





Publishing records..
{'stations': '948702', 'date': '2025-09-25', 'air_temperature_celcius': 23, 'relative_humidity': 58.8, 'windspeed_knots': 9.1, 'max_wind_speed': 15.0, 'precipitation': '0.01G', 'GHI_w/m2': 183, 'producer': 'Producer_1'}
{'stations': '948702', 'date': '2025-09-25', 'air_temperature_celcius': 23, 'relative_humidity': 58.8, 'windspeed_knots': 9.1, 'max_wind_speed': 15.0, 'precipitation': '0.01G', 'GHI_w/m2': 183, 'producer': 'Producer_1'}
{'stations': '948702', 'date': '2025-09-26', 'air_temperature_celcius': 18, 'relative_humidity': 55.6, 'windspeed_knots': 8.2, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 147, 'producer': 'Producer_1'}
{'stations': '948702', 'date': '2025-09-26', 'air_temperature_celcius': 18, 'relative_humidity': 55.6, 'windspeed_knots': 8.2, 'max_wind_speed': 12.0, 'precipitation': '0.00I', 'GHI_w/m2': 147, 'producer': 'Producer_1'}
{'stations': '948701', 'date': '2025-09-27', 'air_temperature_celcius': 20, 'relative_humidity': 62

{'stations': '948701', 'date': '2025-10-14', 'air_temperature_celcius': 14, 'relative_humidity': 47.9, 'windspeed_knots': 5.3, 'max_wind_speed': 9.9, 'precipitation': '0.00I', 'GHI_w/m2': 122, 'producer': 'Producer_1'}
{'stations': '948701', 'date': '2025-10-14', 'air_temperature_celcius': 14, 'relative_humidity': 47.9, 'windspeed_knots': 5.3, 'max_wind_speed': 9.9, 'precipitation': '0.00I', 'GHI_w/m2': 122, 'producer': 'Producer_1'}
{'stations': '948701', 'date': '2025-10-15', 'air_temperature_celcius': 9, 'relative_humidity': 39.0, 'windspeed_knots': 5.6, 'max_wind_speed': 8.9, 'precipitation': '0.00G', 'GHI_w/m2': 84, 'producer': 'Producer_1'}
{'stations': '948701', 'date': '2025-10-15', 'air_temperature_celcius': 9, 'relative_humidity': 39.0, 'windspeed_knots': 5.6, 'max_wind_speed': 8.9, 'precipitation': '0.00G', 'GHI_w/m2': 84, 'producer': 'Producer_1'}
{'stations': '948701', 'date': '2025-10-16', 'air_temperature_celcius': 22, 'relative_humidity': 52.8, 'windspeed_knots': 9.1, '

In [ ]:
"""
Event Producer 2
"""
hotspot_AQUA = pd.read_csv("./hotspot_AQUA_streaming.csv", sep=',')


In [41]:
hotspot_AQUA.head()

,latitude,longitude,confidence,surface_temperature_celcius
0,-37.623,149.323,51,38
1,-38.038,142.986,65,40
2,-37.950,142.366,92,70
3,-38.231,147.172,83,60
4,-37.903,145.250,53,44


In [58]:
"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Scenaries02'
    print('Publishing records..')
    producer02 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 10
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_AQUA_row in hotspot_AQUA.iterrows():
        hotspot_AQUA_row = hotspot_AQUA.loc[rand_row_indices[index]]

        hotspot_AQUA_doc = {
            'latitude': hotspot_AQUA_row['latitude'],
            'longtitude': hotspot_AQUA_row['longitude'],
            'confidence': hotspot_AQUA_row['confidence'],
            'surface_temperature_celcius': hotspot_AQUA_row['surface_temperature_celcius'],
            'producer': "Producer_2",
            'random_time': rand_row_indices[index]%max_time + 1 # at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        print(hotspot_AQUA_doc['random_time'])
        publish_msg(producer01, topic, hotspot_AQUA_doc)
        # every random seconds
        sleep(hotspot_AQUA_doc['random_time']) 



Publishing records..
1
{'latitude': -35.7632, 'longtitude': 142.5059, 'confidence': 72.0, 'surface_temperature_celcius': 47.0, 'producer': 'Producer_2', 'random_time': 1}
{'latitude': -35.7632, 'longtitude': 142.5059, 'confidence': 72.0, 'surface_temperature_celcius': 47.0, 'producer': 'Producer_2', 'random_time': 1}
3
{'latitude': -36.0966, 'longtitude': 142.3635, 'confidence': 86.0, 'surface_temperature_celcius': 61.0, 'producer': 'Producer_2', 'random_time': 3}
{'latitude': -36.0966, 'longtitude': 142.3635, 'confidence': 86.0, 'surface_temperature_celcius': 61.0, 'producer': 'Producer_2', 'random_time': 3}
7
{'latitude': -37.329, 'longtitude': 143.136, 'confidence': 89.0, 'surface_temperature_celcius': 62.0, 'producer': 'Producer_2', 'random_time': 7}
{'latitude': -37.329, 'longtitude': 143.136, 'confidence': 89.0, 'surface_temperature_celcius': 62.0, 'producer': 'Producer_2', 'random_time': 7}


KeyboardInterrupt: 

In [49]:
"""
Event Producer 3
"""
hotspot_TERRA= pd.read_csv("./hotspot_TERRA_streaming.csv", sep=',')


In [50]:
hotspot_TERRA.head()

,latitude,longitude,confidence,surface_temperature_celcius
0,-37.966,145.051,78,68
1,-35.541,143.311,82,63
2,-35.554,143.307,67,53
3,-35.543,143.316,86,67
4,-37.708,145.100,80,54


In [ ]:

"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Scenaries03'
    print('Publishing records..')
    producer03 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 10
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_TERRA_row in hotspot_TERRA.iterrows():
        hotspot_TERRA_row = hotspot_TERRA.loc[rand_row_indices[index]]

        hotspot_AQUA_doc = {
            'latitude': hotspot_TERRA_row['latitude'],
            'longtitude': hotspot_TERRA_row['longitude'],
            'confidence': hotspot_TERRA_row['confidence'],
            'surface_temperature_celcius': hotspot_TERRA_row['surface_temperature_celcius'],
            'producer': "Producer_3",
            'random_time': rand_row_indices[index]%max_time + 1 # at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        print(hotspot_AQUA_doc['random_time'])
        publish_msg(producer01, topic, hotspot_AQUA_doc)
        # every random seconds
        sleep(hotspot_AQUA_doc['random_time']) 

